# What I did

At its core, this module allows the user to import images, modify them in various ways, while viewing their progress, and save the result to a new image file. But how did I do that?

## Image Storage



One of the first questions I considered was how to store the images within the glitcher object. There was no singular format that would work for everything I wanted to be able to do to the images. 
- For some methods, such as shuffling the bytes of the image file, the image needed to be encoded in a format such as JPEG. 
- For other methods, such as changing the contrast, I wanted to take advantage of the Python Image Library (PIL), which required the image representation to be stored as a PIL Image object. 
- For the bulk of my methods, I wanted to be able to interact with the pixels of the image as plainly as possible, which meant storing it as a three dimensional Numpy array: two dimensions for the width and height, and the third dimension for the RGB values of each pixel. Since I was writing this in Python, it was important to use a package such as Numpy, since for most actions, manipulating arrays in Numpy is significantly faster than manipulating them item by item as Python lists.

To handle these various storage methods, and the conversion between them, I created a new class, called ImageStorage, that would handle all the conversion. When the Glitcher needed the image to be in that format, it would call call the ImageStorage method to fetch the image in that format. If the image was already in that format, the ImageStorage object would just return that image. Otherwise, it would conver to the needed format.

But between Numpy arrays, PIL Images, and bytes for JPEG, PNG, and BMP image encodings, we now have $5$ formats, and we need to be able to convert from any one of these methods to any other one. If we need a separate function to convert from each method to each other method, we're looking at an unwieldy amount of code. Additionally, if we add a new format— say, we want to be able to work with GIF encoded images— we don't want to have to write separate methods to convert from that format to every other format.



<img src="img/conversion_problem.jpg" alt="All formats, with double arrows connecting each to each other. Very cluttered." style="width: 500px;"/>


To solve this problem, we use PIL Image objects as a kind of middle ground. There are already built in methods for converting between PIL Images and numpy arrays. PIL is built to be able to read and write image files, so we can modify that easily to allow it to read and write to byte objects instead. Now, when converting between two representations, we just convert to a PIL image, and then to the other format.

<img src="img/conversion_solution.jpg" alt="The formats go through PIL, now." style="width: 500px;"/>

## Checkpoints

I wanted playing with Glitcher to feel freeing. Glitches are generally associated with things breaking, and so can be tense experiences. With this project, however, I wanted to create code that allowed people to experiment without  anxiety.

To make that happen, it was important to have a way to undo actions. I implemented this as a list of copied Glitcher objects— essentially just a shell around the corresponding ImageStorage representaiton. To set a checkpoint, the code just makes a new copy and pushes it on the end of the list. To undo to the most recent checkpoint, it pops the last element off the list, and sets that objects image as the current image.

To make it easier to go back a large number of checkpoints, I give the user an option of giving checkpoints a name, which is stored alongside the copy in the list. When the user reverts to a named checkpoint, the code searches through the list for that name, discards all checkpoints after the name, and sets that checkpoint image as the current image.

There is also a method for making a copy of a Glitcher object. As it stands, the copy does not have the checkpoints of the original, although I may implement that in the future.

## Cellular Automata

I *love* cellular automata. Essentially, you have an array of pixels, whether one- or two-dimensional. Each pixel either has a value of 1 or 0. For each pixel, you look at it's value and the value of its neighbors, and use a specific rule to determine what it's value should be 1 or 0 in the next generation. 

## Playing with Image Encoding

This is the part where it gets a little rough. A lot of great glitch art is created by opening images in a text editor, and manipulating them as text. Image encodings are complicated, and I found editing them to be one of the most unpredictable parts of this project. 

To compound that, it is very easy to change the data in such a way that programs will be unable to display them. I found that some images would not display in the Preview app (the default image viewer on Apple computers), but would display when opened in Google Chrome. This was especially the case with .bmp images, since that format is primarily supported on Microsoft Windows computers. Some changes to the file rendered them unreadable in any program.